<a href="https://colab.research.google.com/github/Alerza/nlp_task/blob/main/GPT2_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install tokenizers

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tokenizers import  Tokenizer , models , normalizers , pre_tokenizers , processors , trainers , decoders

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [4]:
decoder = decoders.ByteLevel()

In [5]:
text = "شب چهارم دیگه طاقت نیاوردم یه فریدون فروغی زدمو وسطای کورش یغمایی بودم"
output = tokenizer.encode(text)
#output = tokenizer.encode("اوردم")
print(output)

outputtokens = tokenizer.convert_ids_to_tokens(output)
print(outputtokens)

#print(tokenizer(text).tokens())

print(decoder.decode(outputtokens[:3]))

[148, 112, 39848, 220, 150, 228, 29519, 12919, 26897, 25405, 17550, 107, 151, 234, 150, 107, 29519, 17550, 115, 12919, 149, 224, 41486, 18923, 228, 151, 234, 12919, 30335, 26897, 38843, 25405, 220, 151, 234, 29519, 18923, 223, 26897, 151, 234, 38843, 30335, 23338, 18923, 223, 26897, 30335, 148, 118, 151, 234, 17550, 110, 38843, 25405, 30335, 42092, 45692, 148, 115, 12919, 151, 234, 220, 150, 102, 30335, 26897, 148, 112, 220, 151, 234, 148, 118, 25405, 12919, 151, 234, 151, 234, 17550, 101, 30335, 38843, 25405]
['Ø', '´', 'Ø¨', 'Ġ', 'Ú', 'Ĩ', 'Ùĩ', 'Ø§', 'Ø±', 'Ùħ', 'ĠØ', '¯', 'Û', 'Į', 'Ú', '¯', 'Ùĩ', 'ĠØ', '·', 'Ø§', 'Ù', 'Ĥ', 'Øª', 'ĠÙ', 'Ĩ', 'Û', 'Į', 'Ø§', 'ÙĪ', 'Ø±', 'Ø¯', 'Ùħ', 'Ġ', 'Û', 'Į', 'Ùĩ', 'ĠÙ', 'ģ', 'Ø±', 'Û', 'Į', 'Ø¯', 'ÙĪ', 'ÙĨ', 'ĠÙ', 'ģ', 'Ø±', 'ÙĪ', 'Ø', 'º', 'Û', 'Į', 'ĠØ', '²', 'Ø¯', 'Ùħ', 'ÙĪ', 'ĠÙĪ', 'Ø³', 'Ø', '·', 'Ø§', 'Û', 'Į', 'Ġ', 'Ú', '©', 'ÙĪ', 'Ø±', 'Ø', '´', 'Ġ', 'Û', 'Į', 'Ø', 'º', 'Ùħ', 'Ø§', 'Û', 'Į', 'Û', 'Į', 'ĠØ', '¨', 'ÙĪ', 'Ø¯', 'Ùħ']
شب


In [6]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""

input_txt = ' رئیس اتحادیه نانوایان'
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length,
                               do_sample=False,
                               pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_greedy[0]))

 رئیس اتحادیه نانوایانی بحدیه بحدیه بحدیه بحدیه بحدیه بحدیه بحدیه بحدیه بحدیه بحدیه بحدیه بحدیه ب�


In [7]:
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings 

In [8]:
position_embeddings.shape

torch.Size([1024, 768])

In [31]:
import numpy as np

text_index1 = tokenizer.encode('man')
embd1 = torch.squeeze(model.transformer.wte.weight[text_index1,:])
#pembd = model.transformer.wpe.weight[text_index,:]
print(embd1.shape)

text_index2 = tokenizer.encode('w')
embd2 = torch.squeeze(model.transformer.wte.weight[text_index2,:])
#pembd = model.transformer.wpe.weight[text_index,:]
#print(embd2)


print((torch.dot(embd1,embd2)).item())

torch.Size([768])
3.8177289962768555


In [83]:
import pandas as pd
a = torch.squeeze(model.get_input_embeddings().weight.clone()).detach().numpy()
px = pd.DataFrame(index=range(tokenizer.vocab_size) )

def convert_ids_to_tokens(i):
    outputtokens = tokenizer.convert_ids_to_tokens([i])
    return decoder.decode(outputtokens)


px['token'] = px.index.to_series().apply(convert_ids_to_tokens, convert_dtype=True, args=())
px

,token
0,!
1,""""
2,#
3,$
4,%
...,...
50252,regress
50253,Collider
50254,informants
50255,gazed


In [84]:
text_indexbase = tokenizer.encode('person')
#text_indexbase = 39848
embdbase = torch.squeeze(model.transformer.wte.weight[text_indexbase,:])
print(embdbase.shape)
max=0

def calc_similarity(i):
    embd = torch.squeeze(model.transformer.wte.weight[i,:])
    smltry = (torch.dot(embd,embdbase)).item()
    return smltry

px['similarity'] = px.index.to_series().apply(calc_similarity, convert_dtype=True, args=())

px.sort_values(['similarity'], ascending=False).head(20)

torch.Size([768])


,token,similarity
39848,ب,16.676958
38843,د,11.696240
45692,س,11.601573
25405,م,11.548094
41486,ت,11.263893
23525,ال,11.143570
44690,ع,11.074633
34247,ا�,11.000047
30335,و,10.751839
148,�,10.680145
